In [1]:
import pandas as pd
import codecs
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime

path = u"C:/Users/Administrator/git/NILM/A题"
yd1 = pd.ExcelFile(path+ u"部分数据/附件1/YD1.xlsx")
yd2 = pd.ExcelFile(path+ u"部分数据/附件1/YD2.xlsx")
yd3 = pd.ExcelFile(path+ u"部分数据/附件1/YD3.xlsx")
yd4 = pd.ExcelFile(path+ u"部分数据/附件1/YD4.xlsx")
yd5 = pd.ExcelFile(path+ u"部分数据/附件1/YD5.xlsx")
yd6 = pd.ExcelFile(path+ u"部分数据/附件1/YD6.xlsx")
yd7 = pd.ExcelFile(path+ u"部分数据/附件1/YD7.xlsx")
yd8 = pd.ExcelFile(path+ u"部分数据/附件1/YD8.xlsx")
yd9 = pd.ExcelFile(path+ u"部分数据/附件1/YD9.xlsx")
yd10 = pd.ExcelFile(path+ u"部分数据/附件1/YD10.xlsx")
yd11 = pd.ExcelFile(path+ u"部分数据/附件1/YD11.xlsx")
yd3yd7 = pd.ExcelFile(path+ u"部分数据/附件3/设备组1-YD3+YD7.xlsx")
yd1yd2yd4 = pd.ExcelFile(path+ u"部分数据/附件3/设备组2-YD1+YD2+YD4.xlsx")
yd4yd5yd10yd11=pd.ExcelFile(path+ u"部分数据/附件3/设备组3-YD4+YD5+YD8+YD10+YD11.xlsx")
yd=[yd1,yd2,yd3,yd4,yd5,yd6,yd7,yd8,yd9,yd10, yd11, yd3yd7, yd1yd2yd4, yd4yd5yd10yd11]

equip_data = []
period_data = []
harmonic_data = []
oper_data = []
for i in range(0,14):
    equip_data.append(yd[i].parse(u'设备数据'));
    period_data.append(yd[i].parse(u'周波数据'));
    harmonic_data.append(yd[i].parse(u'谐波数据'));
for i in range(0,11):
    oper_data.append(yd[i].parse(u'操作记录'));


In [2]:
### The function to detect and characterize pulses.

def characterize_pulse(array, posi_thresh, nega_thresh):
    # author - D.Q. Huang 20180406
    # this function is used to find and characterize pulse in each instrument
    # input:
    #       - instrument data array
    #       - positive threshold
    #       - negative threshold
    # output:
    pulse_wf = [] # pulse waveform, this is not part of output
    pulse_area = [] # pulse area
    pulse_trig_t = [] # time of first point that is triggered
    pulse_heith_t = [] # height point time
    pulse_end_t = [] # time of last point that is triggered
    pulse_h = [] # pulse height
    
    wf = np.array([0] + list(np.diff(array)))
    time_s = np.array(range(len(wf)))
    cut_p = (wf > posi_thresh)
    cut_p_diff = np.diff(cut_p*1) # convert boolean to number 
    pp_start_idx = np.where(cut_p_diff == 1)[0] + 1 # first index cutted out by diff - python index (starts with 0)
    pp_end_idx = np.where(cut_p_diff == -1)[0] # we don't need +1 here somehow - python index (starts with 0)

    for i in range(0, min([len(pp_start_idx),len(pp_end_idx)])):
        temp_pulse_wf = wf[np.array(range(pp_start_idx[i],pp_end_idx[i]+1))]
        temp_pulse_time_s = time_s[np.array(range(pp_start_idx[i],pp_end_idx[i]+1))]
        pulse_wf.append(temp_pulse_wf)
        pulse_area.append(temp_pulse_wf.sum())
        pulse_h.append(temp_pulse_wf.max())
        pulse_trig_t.append(temp_pulse_time_s[0])
        cut_pulse_heigh_t = np.where(temp_pulse_wf == temp_pulse_wf.max())
        pulse_heith_t.append(temp_pulse_time_s[cut_pulse_heigh_t[0][0]])
        pulse_end_t.append(temp_pulse_time_s[-1])
        
    cut_n = (wf < nega_thresh)
    cut_n_diff = np.diff(cut_n*1)
    np_start_idx = np.where(cut_n_diff == 1)[0] + 1
    np_end_idx = np.where(cut_n_diff == -1)[0]
    
    for i in range(0, min([len(np_start_idx),len(np_end_idx)])):
        temp_pulse_wf = wf[np.array(range(np_start_idx[i],np_end_idx[i]+1))]
        temp_pulse_time_s = time_s[np.array(range(np_start_idx[i],np_end_idx[i]+1))]
        pulse_wf.append(temp_pulse_wf)
        pulse_area.append(temp_pulse_wf.sum())
        pulse_h.append(temp_pulse_wf.min())
        pulse_trig_t.append(temp_pulse_time_s[0])
        cut_pulse_heigh_t = np.where(temp_pulse_wf == temp_pulse_wf.min())
        pulse_heith_t.append(temp_pulse_time_s[cut_pulse_heigh_t[0][0]])
        pulse_end_t.append(temp_pulse_time_s[-1])
        
    pulse_area = np.array(pulse_area)
    pulse_trig_t = np.array(pulse_trig_t)
    pulse_heith_t = np.array(pulse_heith_t)
    pulse_end_t = np.array(pulse_end_t)
    pulse_h = np.array(pulse_h)
    
    index = np.argsort(pulse_trig_t)
    pulse_area = pulse_area[index]
    pulse_trig_t = pulse_trig_t[index]
    pulse_heith_t = pulse_heith_t[index]
    pulse_end_t = pulse_end_t[index]
    pulse_h = pulse_h[index]
    
    return [pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h] 

### We use P values for identifying pulses, i.e. operations.

In [3]:
pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h = characterize_pulse(equip_data[2]['P'].values, posi_thresh=20, nega_thresh=-20)

In [4]:
pulse_area

array([ 17265,    -58,    -23,    -31,    -22,    -24,    -21,    -22,
       -16997], dtype=int64)

In [5]:
pulse_heith_t

array([ 54,  55,  60,  62,  66,  68, 107, 111, 129])

In [6]:
pulse_trig_t

array([ 53,  55,  60,  62,  66,  68, 107, 111, 128])

In [7]:
pulse_end_t

array([ 54,  56,  60,  62,  66,  68, 107, 111, 129])

In [8]:
pulse_h

array([ 15209,    -30,    -23,    -31,    -22,    -24,    -21,    -22,
       -14826], dtype=int64)

# The charaterize_pulse function detected the two most significant pulses. The rest small pulses are just fluctuations which are not useful for identifying yd3. However, these small pulses might be distractive when identifying other equipments. 

### Let's defined the equipment detector for yd3

In [9]:
def yd3_detector(pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h):
    '''
    Input : pulse_area, pulse_trig_t, pulse_heith_t, pulse_end_t, pulse_h,  i.e.
            The characteristics of the pulses identified by the characterize_pulse function. 
    Output : Exist: boolean. If True, then yd3 is decteted. If False, then yd3 is not detected.
             operation : list of tuples. Each tuple contains two values: one is the operation, the other is the operation time.
             mask : numpy array that put False at the location where yd3 pulse is .
    '''
    criteria = 15000 
    index = np.where(np.abs(pulse_area)>criteria)[0] # Find the index of values bigger than criteria
    mask = np.ones(len(pulse_area), dtype=bool)
    operation = []
    if len(index)==0: # i.e. yd3 is not detected
        exist = False
    else:
        exist = True
        for i in index:
            if pulse_area[i]>0:
                operation.append(('yd3 turn on', pulse_trig_t[i]))
            if pulse_area[i]<0:
                operation.append(('yd3 turn off', pulse_trig_t[i]))
        mask[index] = False
    return exist, operation , mask

In [10]:
exist, operation, mask = yd3_detector(pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h)

In [11]:
exist, operation, mask

(True,
 [('yd3 turn on', 53), ('yd3 turn off', 128)],
 array([False,  True,  True,  True,  True,  True,  True,  True, False], dtype=bool))

### Let's define the function for masking the detected pulses.

In [12]:
def mask_process(mask,pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h):
    '''
    Input : mask,pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h
    Output : values of pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h after being masked
    '''
    pulse_area = pulse_area[mask]
    pulse_trig_t = pulse_trig_t[mask]
    pulse_heith_t = pulse_heith_t[mask]
    pulse_end_t = pulse_end_t[mask]
    pulse_h = pulse_h[mask]
    
    return pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h


In [13]:
pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h=mask_process(mask,pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h)

In [14]:
pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h

(array([-58, -23, -31, -22, -24, -21, -22], dtype=int64),
 array([ 55,  60,  62,  66,  68, 107, 111]),
 array([ 55,  60,  62,  66,  68, 107, 111]),
 array([ 56,  60,  62,  66,  68, 107, 111]),
 array([-30, -23, -31, -22, -24, -21, -22], dtype=int64))

### Let's test our yd3_detector on other equip_data. It works for the moment.

In [15]:
pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h = characterize_pulse(equip_data[13]['P'].values, posi_thresh=20, nega_thresh=-20)

In [16]:
yd3_detector(pulse_area,pulse_trig_t,pulse_heith_t,pulse_end_t,pulse_h)

(False,
 [],
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 